# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770275530965                   -0.53    8.0         
  2   -2.771681325589       -2.85       -1.30    1.0    160ms
  3   -2.771714059053       -4.49       -2.65    1.0    122ms
  4   -2.771714712673       -6.18       -4.01    2.0    196ms
  5   -2.771714715029       -8.63       -4.37    2.0    143ms
  6   -2.771714715239       -9.68       -5.24    1.0    127ms
  7   -2.771714715249      -11.00       -5.80    2.0    158ms
  8   -2.771714715250      -12.62       -6.66    1.0    135ms
  9   -2.771714715250      -13.58       -7.65    2.0    169ms
 10   -2.771714715250      -14.27       -7.65    2.0    161ms
 11   -2.771714715250   +  -13.78       -9.60    1.0    142ms


-0.00013457359496073977

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770542506269                   -0.53    9.0         
  2   -2.771779087654       -2.91       -1.32    1.0    124ms
  3   -2.771801722082       -4.65       -2.50    1.0    164ms
  4   -2.771802015375       -6.53       -3.26    1.0    160ms
  5   -2.771802073415       -7.24       -3.95    2.0    145ms
  6   -2.771802074263       -9.07       -4.36    1.0    153ms
  7   -2.771802074470       -9.68       -5.26    1.0    131ms
  8   -2.771802074476      -11.22       -5.95    2.0    164ms
  9   -2.771802074476      -12.84       -6.75    1.0    140ms
 10   -2.771802074476   +  -14.65       -6.91    2.0    184ms
 11   -2.771802074476      -15.05       -7.61    1.0    144ms
 12   -2.771802074476   +  -14.51       -8.24    2.0    171ms


0.01761222148774927

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457359496073977
Displaced dipole:  0.01761222148774927
Polarizability :   1.7746795082710007


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920875771e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551664816e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852768283959e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602723014e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088786652754e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.277478619868e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 8.120313626312e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 8.120313626312e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 6.639307839539e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.044371116531e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.266059106497e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.266059106497e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.2656974471358228e-13
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.